In [15]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.callbacks import EarlyStopping



In [16]:
data = pd.read_csv("Churn_Modelling.csv")
data.drop(columns=["RowNumber", "CustomerId", "Surname"], inplace=True,axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_encoder.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder.get_feature_names_out(['Geography']))
data = pd.concat([data, geo_encoded_df], axis=1)
data.drop(columns=['Geography'], inplace=True)

x = data.drop(columns=['Exited'], axis=1)
y = data[['Exited']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

with open("scaler_hp.pkl", "wb") as f:
    pickle.dump(scaler, f)

with open("ohe_hp.pkl", "wb") as f:
    pickle.dump(one_hot_encoder, f)

with open("label_encoder_hp.pkl", "wb") as f:
    pickle.dump(label_encoder_gender, f)




In [17]:
# Defining a function to create the model and try different params using GridSearchCV

def create_model(neurons=16, layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(x_train.shape[1],)))
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [18]:
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0)


In [21]:
param_grid = {
    "model__neurons": [8, 16, 32],
    "model__layers": [1, 2, 3],
    "epochs": [50, 100]
}


In [22]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid_search.fit(x_train, y_train)

print("Best params:", grid_result.best_params_)
print("Best score:", grid_result.best_score_)


/Users/madhav.khurana@grofers.com/Documents/DS2025/DL_Project1/venv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
/Users/madhav.khurana@grofers.com/Documents/DS2025/DL_Project1/venv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
/Users/madhav.khurana@grofers.com/Documents/DS2025/DL_Project1/venv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
/Users/madhav.khurana@grofers.com/Documents/DS2025/DL_Project1/venv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not

KeyboardInterrupt: 